# Most Used Functions in DiscoGAN

DiscoGAN is a type of Generative Adversarial Network (GAN) designed to learn cross-domain relationships. It aims to discover and translate relationships between different domains, such as transferring styles from one domain to another. In this notebook, we will cover some of the most commonly used functions and techniques for implementing a simplified version of DiscoGAN using TensorFlow and Keras.

## 1. Building the Generator

The generator in DiscoGAN learns to translate images from one domain to another. It uses a series of convolutional layers and residual blocks.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Activation, Add, BatchNormalization
from tensorflow.keras.models import Model

# Define the residual block
def residual_block(x, filters):
    res = Conv2D(filters, (3, 3), padding='same')(x)
    res = BatchNormalization()(res)
    res = Activation('relu')(res)
    res = Conv2D(filters, (3, 3), padding='same')(res)
    res = BatchNormalization()(res)
    return Add()([res, x])

# Function to build the generator
def build_generator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (7, 7), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    for _ in range(6):
        x = residual_block(x, 256)
    x = Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(3, (7, 7), padding='same')(x)
    x = Activation('tanh')(x)
    return Model(inputs, x)

# Instantiate and summarize the generator
generator = build_generator()
generator.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 64, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 64, 64, 64)     │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 64, 64, 64)     │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 64, 64, 64)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 32, 32, 128)    │         73,856 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 32, 32, 128)    │            512 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 32, 32, 128)    │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 16, 16, 256)    │        295,168 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 16, 16, 256)    │          1,024 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 16, 16, 256)    │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 16, 16, 256)    │        590,080 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 16, 16, 256)    │          1,024 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 16, 16, 256)    │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 16, 16, 256)    │        590,080 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 16, 16, 256)    │          1,024 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 16, 16, 256)    │              0 │ batch_normalization_4… │
│                           │                        │                │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)    

 Total params: 7,852,547 (29.96 MB)

 Trainable params: 7,845,123 (29.93 MB)

 Non-trainable params: 7,424 (29.00 KB)

## 2. Building the Discriminator

The discriminator in DiscoGAN learns to distinguish between real images and fake images generated by the generator. It uses a series of convolutional layers to classify the images.

In [3]:
from tensorflow.keras.layers import LeakyReLU, Flatten, Dense

# Function to build the discriminator
def build_discriminator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(512, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs, x)

# Instantiate and summarize the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 32, 32, 64)          │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 16, 16, 128)         │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 8, 8, 256)           │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 8, 8, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 4, 4, 512)           │       2,097,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 4, 4, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           8,193 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,768,321 (10.56 MB)

 Trainable params: 2,766,529 (10.55 MB)

 Non-trainable params: 1,792 (7.00 KB)

## 3. Building the DiscoGAN

The DiscoGAN combines two generators and two discriminators. One generator translates images from domain A to domain B, and the other translates images from domain B to domain A. The cycle-consistency loss ensures that an image translated to the other domain and back results in the original image.

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to build the DiscoGAN
def build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B):
    # Make discriminators non-trainable for adversarial training
    discriminator_A.trainable = False
    discriminator_B.trainable = False

    # Input images from both domains
    input_A = Input(shape=(64, 64, 3))
    input_B = Input(shape=(64, 64, 3))

    # Translate images to the other domain
    fake_B = generator_A_to_B(input_A)
    fake_A = generator_B_to_A(input_B)

    # Translate images back to original domain
    reconstructed_A = generator_B_to_A(fake_B)
    reconstructed_B = generator_A_to_B(fake_A)

    # Discriminators determine validity
    valid_A = discriminator_A(fake_A)
    valid_B = discriminator_B(fake_B)

    # Combined model to update generators
    combined = Model(inputs=[input_A, input_B], outputs=[valid_A, valid_B, reconstructed_A, reconstructed_B])
    combined.compile(optimizer=Adam(0.0002, 0.5), loss=['mse', 'mse', 'mae', 'mae'], loss_weights=[1, 1, 10, 10])

    return combined

# Instantiate and summarize the DiscoGAN
generator_A_to_B = build_generator()
generator_B_to_A = build_generator()
discriminator_A = build_discriminator()
discriminator_B = build_discriminator()

discogan = build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B)
discogan.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_6             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, 64, 64, 3)      │      7,852,547 │ input_layer_7[0][0],   │
│                           │                        │                │ functional_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ (None, 64, 64, 3)      │      7,852,547 │ input_layer_6[0][0],   │
│                           │                        │                │ functional_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_4 (Functional) │ (None, 1)              │      2,768,321 │ functional_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_5 (Functional) │ (None, 1)              │      2,768,321 │ functional_2[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,241,736 (81.03 MB)

 Trainable params: 15,690,246 (59.85 MB)

 Non-trainable params: 5,551,490 (21.18 MB)

## 4. Training the DiscoGAN

Training the DiscoGAN involves alternating between training the discriminators and training the generators with cycle-consistency loss.

In [ ]:
# Instantiate the generators and discriminators
generator_A_to_B = build_generator()
generator_B_to_A = build_generator()
discriminator_A = build_discriminator()
discriminator_B = build_discriminator()

# **Key Change**: Compile the discriminators
discriminator_A.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])
discriminator_B.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])

# Instantiate and summarize the DiscoGAN
discogan = build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B)
discogan.summary()

# Sample data
data_A = np.random.rand(100, 64, 64, 3).astype(np.float32)
data_B = np.random.rand(100, 64, 64, 3).astype(np.float32)

# Training parameters
epochs = 10000
batch_size = 1
patch_size = discriminator_A.output_shape[1]

# Training loop
for epoch in range(epochs):
    for _ in range(data_A.shape[0] // batch_size):
        # Train discriminators with real samples
        idx = np.random.randint(0, data_A.shape[0], batch_size)
        X_real_A, y_real_A = data_A[idx], np.ones((batch_size, patch_size, patch_size, 1))
        X_real_B, y_real_B = data_B[idx], np.ones((batch_size, patch_size, patch_size, 1))
        dA_loss_real = discriminator_A.train_on_batch(X_real_A, y_real_A)
        dB_loss_real = discriminator_B.train_on_batch(X_real_B, y_real_B)

        # Train discriminators with fake samples
        X_fake_A, y_fake_A = generator_B_to_A.predict(X_real_B), np.zeros((batch_size, patch_size, patch_size, 1))
        X_fake_B, y_fake_B = generator_A_to_B.predict(X_real_A), np.zeros((batch_size, patch_size, patch_size, 1))
        dA_loss_fake = discriminator_A.train_on_batch(X_fake_A, y_fake_A)
        dB_loss_fake = discriminator_B.train_on_batch(X_fake_B, y_fake_B)

        # Train generators
        g_loss = discogan.train_on_batch([X_real_A, X_real_B], [y_real_A, y_real_B, X_real_A, X_real_B])

    # Summarize the loss for this epoch
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1}/{epochs}, dA_real_loss={dA_loss_real[0]}, dA_fake_loss={dA_loss_fake[0]}, dB_real_loss={dB_loss_real[0]}, dB_fake_loss={dB_loss_fake[0]}, g_loss={g_loss[0]}')


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_18            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_13             │ (None, 64, 64, 3)      │      7,852,547 │ input_layer_19[0][0],  │
│ (Functional)              │                        │                │ functional_12[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_12             │ (None, 64, 64, 3)      │      7,852,547 │ input_layer_18[0][0],  │
│ (Functional)              │                        │                │ functional_13[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_14             │ (None, 1)              │      2,768,321 │ functional_13[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_15             │ (None, 1)              │      2,768,321 │ functional_12[0][0]    │
│ (Functional)              │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,241,736 (81.03 MB)

 Trainable params: 15,690,246 (59.85 MB)

 Non-trainable params: 5,551,490 (21.18 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step
